
To check cam view while trainng RL model,
make sure that train.ipynb is running


In [1]:
import sys
sys.path.append('/home/kaustubh/Documents/GitHub/Self_Driving_Car/')

In [2]:
import carla
import numpy as np
from Imitation_Learning_RL.process_frame import process_frame

['', '/opt/ros/humble/lib/python3.10/site-packages', '/opt/ros/humble/local/lib/python3.10/dist-packages', '/home/kaustubh/miniconda3/envs/carla_15/lib/python310.zip', '/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10', '/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/lib-dynload', '/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL/model', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL/model', '/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/setuptools/_vendor', '/tmp/tmp_bc_0dep']


/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
seg_image = None
rgb_image = None
collision = None
lane_invasion = None

In [ ]:
# Callbacks
def image_callback(data):
    global rgb_image, seg_image
    array = np.frombuffer(data.raw_data, dtype=np.uint8)
    array = np.reshape(array, (data.height, data.width, 4))[:, :, :3]
    rgb_image = array
    seg_image = process_frame(rgb_image)

def collision_callback(data):
    global collision
    collision = True

def lane_invasion_callback(event):
    global lane_invasion
    lane_invasion = True

In [5]:
client_ = carla.Client('localhost', 2000)
client_.set_timeout(10.0)
world_ = client_.get_world()
blueprint_library_ = world_.get_blueprint_library()

In [6]:
vehicle = world_.get_actors().filter("vehicle.volkswagen.t2_2021")
camera_sensor = world_.get_actors().filter("sensor.camera.rgb")[0]
collision_sensor = world_.get_actors().filter("sensor.other.collision")[0]
lane_invasion_sensor = world_.get_actors().filter("sensor.other.lane_invasion")[0]

In [ ]:
camera_sensor.listen(lambda data: image_callback(data))
collision_sensor.listen(lambda data: collision_callback(data))
lane_invasion_sensor.listen(lambda data: lane_invasion_callback(data))

In [9]:
for sensors in world_.get_actors().filter("*sensor*"):
    print(sensors)

Actor(id=29, type=sensor.camera.rgb)
Actor(id=31, type=sensor.other.lane_invasion)
Actor(id=30, type=sensor.other.collision)


In [ ]:
while True